In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nb
import pandas as pd
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [3]:
def jo2allen_vol(data):
    data = np.swapaxes(np.swapaxes(data, 0,1), 1,2)
    data = np.flip(np.flip(data,1),0)
    return data

In [4]:
annot = np.array(nb.load('/home/julia/data/gradients/atlas/allen_api/annotation.nii.gz').get_data(), dtype='float64')
gradient = nb.load('/home/julia/data/gradients/results/gradient0000_highres.nii.gz').get_data()
mask = nb.load('/home/julia/data/gradients/atlas/cortex/cortex_mask_25um.nii.gz').get_data()

In [5]:
mcc = MouseConnectivityCache(manifest_file=
                             '/home/julia/data/gradients/atlas/allen_api/mouse_connectivity_manifest.json')
structure_tree = mcc.get_structure_tree()
structure_map = structure_tree.get_name_map()

In [6]:
gradient = jo2allen_vol(gradient)
mask = jo2allen_vol(mask)
gradient[mask==0] = np.nan
annot[mask==0] = np.nan

In [8]:
unique_annot = np.unique(annot)
unique_annot = unique_annot[np.isnan(unique_annot)==0]
unique_annot = unique_annot[1:]

In [9]:
region_means = []
region_names=[]
for region in range(unique_annot.shape[0]):
    region_means.append(np.mean(gradient[annot==unique_annot[region]]))
    region_names.append(structure_map[unique_annot[region]])

In [12]:
df = pd.DataFrame(data=zip(unique_annot, region_names, region_means), 
                  columns=['idx', 'name', 'mean'])

In [20]:
df = df.sort_values(by=['mean'])

In [22]:
df.to_csv('/home/julia/data/gradients/results/region_means_gradient0000.csv')

In [ ]:
ACC
                    S1 trunk, lower limbs
                                                Higher visual
                                                                Higher auditory
                    S1 barrel field
                                                V1
Temporal association
Prelimbic
Hippocampus
                                                                A1
                                                V1
Infralimbic
                                                                                        M2
                    Olfactory bulb
Frontal pole
Agranular insula
                                                                                        M1
                    S1 nose, mouth
                    Gustatory

### Try broader region definitions

In [195]:
parents = []
unique_parents = set()
for child in unique_annot:
    parents.append(structure_tree.parent_ids([child])[0])
    unique_parents.add(structure_tree.parent_ids([child])[0])
unique_parents.remove(None)
unique_parents = list(unique_parents)

In [196]:
parents=np.array(parents)
region_means = np.array(region_means)

In [203]:
parent_means = []
parent_names = []
for parent in unique_parents:
    parent_means.append(np.mean(region_means[parents==parent]))
    parent_names.append(structure_map[parent])

In [204]:
df_broad = pd.DataFrame(data=zip(unique_parents, parent_names, parent_means), 
                        columns=['idx', 'name', 'mean'])

In [206]:
df_broad = df_broad.sort_values(by=['mean'])

In [208]:
df_broad.to_csv('/home/julia/data/gradients/results/parent_means_gradient0000.csv')